In [ ]:
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import GridSearchCV
from pycaret.regression import setup, compare_models
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor, ExtraTreesRegressor
from sklearn.linear_model import BayesianRidge, OrthogonalMatchingPursuit, LinearRegression, Lasso, Ridge
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor

# Read train and test sets

In [ ]:
df_train = pd.read_csv("data/train.csv")
df_test = pd.read_csv("data/test.csv")

# Store both dfs lengths

In [ ]:
m_train = df_train.shape[0]
m_test = df_test.shape[0]
m_train, m_test

# Concat the dfs

In [ ]:
df = pd.concat([df_train, df_test])
assert df.shape[0] == m_train + m_test

In [ ]:
df.head()

# Save target

In [ ]:
target_col, target = "SalePrice", df["SalePrice"]

#  Drop Id and target columns

In [ ]:
cols_to_drop = ["Id", target_col]
df.drop(cols_to_drop, axis=1, inplace=True)

# View and fix columns data-types

In [ ]:
df.select_dtypes(object).columns

In [ ]:
df.select_dtypes(np.number).columns

In [ ]:
num_to_obj_cols = ['MSSubClass', 'MoSold']
df[num_to_obj_cols] = df[num_to_obj_cols].astype(object)

In [ ]:
cols_cat = df.select_dtypes(object).columns.to_list()
cols_num = df.select_dtypes(np.number).columns.to_list()

# Impute categorical columns

In [ ]:
cols_cat_na = df[cols_cat].isnull().sum()[df[cols_cat].isnull().sum() > 0]
cols_cat_na

In [ ]:
mode_filled_cols = ["MSZoning", "Utilities", "Exterior1st", "Exterior2nd", "MasVnrType", "Electrical", "KitchenQual", "Functional", "SaleType"]
for col in mode_filled_cols:
    df[col].fillna(df[col].mode()[0], inplace=True)

none_filled_cols = ["Alley", "BsmtQual", "BsmtCond", "BsmtExposure", "BsmtFinType1", "BsmtFinType2", "FireplaceQu", "GarageType", "GarageFinish", "GarageQual", "GarageCond", "PoolQC", "Fence", "MiscFeature"]
for col in none_filled_cols:
    df[col].fillna("None", inplace=True)
    
df[cols_cat].isnull().sum().sum()

# Change ordinal columns to numeric, and encode accordingly

In [ ]:
df.select_dtypes(object).columns

In [ ]:
oe = OrdinalEncoder(categories=[['Reg', 'IR1', 'IR2', 'IR3']])
df.loc[:, "LotShape"] = oe.fit_transform(df[["LotShape"]])
df["LotShape_sq"] = df["LotShape"] ** 2

In [ ]:
oe = OrdinalEncoder(categories=[['Gtl', 'Mod', 'Sev']])
df.loc[:, "LandSlope"] = oe.fit_transform(df[["LandSlope"]])
df["LandSlope_sq"] = df["LandSlope"] ** 2

In [ ]:
qual_oe = OrdinalEncoder(categories=[['None', 'Po', 'Fa', 'TA', 'Gd', 'Ex']])
for col in ["ExterQual", "ExterCond", 'BsmtQual', 'BsmtCond', 'HeatingQC', 'KitchenQual', 'FireplaceQu', 'GarageQual', 'GarageCond', 'PoolQC']:
    df.loc[:, col] = qual_oe.fit_transform(df[[col]])
    df[col + "sq"] = df[col] ** 2

In [ ]:
oe = OrdinalEncoder(categories=[['None', 'No', 'Mn', 'Av', 'Gd']])
df.loc[:, "BsmtExposure"] = oe.fit_transform(df[["BsmtExposure"]])
df["BsmtExposure_sq"] = df["BsmtExposure"] ** 2

In [ ]:
oe = OrdinalEncoder(categories=[['None', 'Unf', 'LwQ', 'Rec', 'BLQ', 'ALQ', 'GLQ']])
for col in ["BsmtFinType1", "BsmtFinType2"]:
    df.loc[:, col] = oe.fit_transform(df[[col]])
    df[col + "sq"] = df[col] ** 2

In [ ]:
oe = OrdinalEncoder(categories=[['None', 'Unf', 'RFn', 'Fin']])
df.loc[:, "GarageFinish"] = oe.fit_transform(df[["GarageFinish"]])
df["GarageFinishsq"] = df["GarageFinish"] ** 2

In [ ]:
oe = OrdinalEncoder(categories=[['N', 'P', 'Y']])
df.loc[:, "PavedDrive"] = oe.fit_transform(df[["PavedDrive"]])
df["PavedDrivesq"] = df["PavedDrive"] ** 2

In [ ]:
oe = OrdinalEncoder(categories=[['None', 'MnWw', 'GdWo', 'MnPrv', 'GdPrv']])
df.loc[:, "Fence"] = oe.fit_transform(df[["Fence"]])
df["Fencesq"] = df["Fence"] ** 2

In [ ]:
cols_cat = df.select_dtypes(object).columns.to_list()
cols_num = df.select_dtypes(np.number).columns.to_list()

# Impute numerical columns

In [ ]:
cols_num_na = df[cols_num].isnull().sum()[df[cols_num].isnull().sum() > 0]
cols_num_na

In [ ]:
zero_filled_cols = ["BsmtFinSF1", "BsmtFinSF2", "BsmtUnfSF", "TotalBsmtSF", "BsmtFullBath", "BsmtHalfBath", "GarageCars", "GarageArea"]
for col in zero_filled_cols:
    df[col].fillna(0, inplace=True)

In [ ]:
cols_num_na = df[cols_num].isnull().sum()[df[cols_num].isnull().sum() > 0]
cols_num_na

In [ ]:
def catboost_imputer(df, cols_num_na):
    
    # save columns to impute and drop them from df
    cols_to_impute = df[cols_num_na]
    df.drop(cols_num_na, axis=1, inplace=True)
    
    for col in cols_num_na:
        # define X and y
        X, y = df.copy(), cols_to_impute[col]

        # get train and test sets
        train_indexes, test_indexes = ~y.isnull(), y.isnull()
        X_train, X_test = X.loc[train_indexes, :], X.loc[test_indexes, :]
        y_train, y_test = y.loc[train_indexes], y.loc[test_indexes]
        
        model = CatBoostRegressor(max_depth=8, random_seed=10,
                                  subsample=0.65, n_estimators=1000,
                                  cat_features=cols_cat)
        model.fit(X_train, y_train)
        cols_to_impute.loc[test_indexes, col] = model.predict(X_test)
        df[col] = cols_to_impute[col]
    return df

In [ ]:
df = catboost_imputer(df, cols_num_na.index)
df.isnull().sum().sum()

# Feature selection

In [ ]:
def get_del_cols(df):
    # return the columns where no more than one value in the categorical features
    # exists in the test data - those features can be ignored
    cols_to_drop = []
    for col in df.select_dtypes(object).columns:
        col_vals = df[col].unique()
        n_vals = len(col_vals)
        n_irrelevant = 0
        for val in col_vals:
            if val not in df[col][m_train:].values:
                n_irrelevant += 1
        if n_irrelevant >= n_vals - 1:
            cols_to_drop.append(col)
    return cols_to_drop

In [31]:
cols_to_drop = get_del_cols(df)
df.drop(cols_to_drop, axis=1, inplace=True)
for col_to_drop in cols_to_drop:
    cols_cat.remove(col_to_drop)
    print("Dropped ", col_to_drop)

Dropped  Utilities


<!-- # Change ordinal columns to numeric, and encode accordingly -->

# Feature Engineering

## Features from the Internet - prefex FE0_

In [ ]:
df["FE0_HighQualSF"] = df["GrLivArea"]+df["1stFlrSF"] + df["2ndFlrSF"]+0.5*df["GarageArea"]+0.5*df["TotalBsmtSF"]+1*df["MasVnrArea"]

df["FE0_SqFtPerRoom"] = df["GrLivArea"] / (df["TotRmsAbvGrd"] +
                                           df["FullBath"] +
                                           df["HalfBath"] +
                                           df["KitchenAbvGr"])

## My features - prefex FE1_

In [ ]:
n_stories_dict = {
    "1Story": 1.0,
    "1.5Fin": 1.5,
    "1.5Unf": 1.5,
    "2Story": 2.0,
    "2.5Fin": 2.5,
    "2.5Unf": 2.5,
    "SFoyer": 2.0,
    "SLvl": 2.0,
}

df["FE1_n_stories"] = df["HouseStyle"].replace(n_stories_dict)
df["FE1_n_stories"].value_counts()

In [ ]:
df["FE1_age_sold"] = df["YrSold"] - df["YearBuilt"]
df["FE1_age_sold_Remod"] = df["YrSold"] - df["YearRemodAdd"]
df["FE1_GarageYrSold"] = df["YrSold"] - df["GarageYrBlt"]

In [ ]:
df["CentralAir"] = df["CentralAir"].replace({"Y": 1, "N": 0})
df["CentralAir"].value_counts()

In [ ]:
df.select_dtypes(np.number).columns

In [ ]:
df['FE1_total_area_hq'] = df["FE0_HighQualSF"] - df['LowQualFinSF']

df['FE1_Total_Home_Quality'] = df['OverallQual'] * df['OverallCond']
df['FE1_Total_bsmt_Quality'] = df['BsmtQual'] * df['BsmtCond']
df['FE1_Total_garage_Quality'] = df['GarageQual'] * df['GarageCond']

df["FE1_weighted_home_area"] = df["FE1_total_area_hq"] * df['FE1_Total_Home_Quality'] 
df["FE1_weighted_bsmt_area"] = df["TotalBsmtSF"] * df['FE1_Total_bsmt_Quality'] 
df["FE1_weighted_garage_area"] = df["GarageArea"] * df['FE1_Total_garage_Quality']
df["FE1_weighted_n_stories"] = df["FE1_n_stories"] * df['OverallQual']


# Split test and train

In [ ]:
X_train, X_test = df[:m_train], df[m_train:]
y_train, _ = target[:m_train], target[m_train:]
n = X_train.shape[1]
X_train.shape, y_train.shape

In [ ]:
# df_ohe = pd.get_dummies(df)
# df_ohe[target_col] = target
# _ = setup(data=df_ohe[:m_train], target='SalePrice')
# best = compare_models()

# Training - CatBoost Grid Search 

In [ ]:
# cat_features = df.select_dtypes(object).columns.to_list()
# model = CatBoostRegressor(random_state=10, cat_features=cat_features)
# kwargs = {
#     "n_estimators": [50 * n],
#     "max_depth": [8],
#     "subsample": [0.65],
#     "reg_lambda": [0.1], 
# }
# clf = GridSearchCV(model, kwargs, verbose=1, n_jobs=2)
# clf.fit(X_train, y_train)
# print(clf.best_score_)
# print(clf.best_params_)

# CatBoost Params:
    {'max_depth': 8, 'n_estimators': 5500, 'reg_lambda': 0.1, 'subsample': 0.65}

In [ ]:
# clf.best_score_

# Training - CatBoost with Best Parameters

In [ ]:
model = CatBoostRegressor(max_depth=8, random_seed=10,
                          subsample=0.65, n_estimators=5500,
                          cat_features=cols_cat)
model.fit(X_train, y_train)

In [ ]:
print(model.best_score_)
fe = model.get_feature_importance(prettified=True)
fe.head(20)

In [ ]:
sub_name = "data/submission_15.csv"
pd.DataFrame(model.predict(X_test), 
            index=range(1461, len(df)+1), 
            columns=['SalePrice']).reset_index().\
            rename(columns={'index': 'id'}).to_csv(sub_name, index=False)

# Score: 0.21233